---

## Data Analysis

- This file differs from [2_data_analysis_1_base_data.ipynb](2_data_analysis_1_base_data.ipynb) in that it:
    - scales the base cleaned data created in [1_data_cleaning.ipynb](1_data_cleaning.ipynb).

Source dataset: 247076 rows × 37 columns
Processed and analyzed dataset: 247076 rows × 37 columns

---

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config

sys.path.insert(1, '../pkgs')
import ml_analysis as mlanlys
import ml_clean_feature as mlclean

import warnings
warnings.filterwarnings("ignore")

---

## 1. Read the cleaned dataset from file

---

In [2]:
year                        = config.year

clean_file                  = config.clean_file
# performance_report          = config.performance_report

# report_path                 = config.report_path
# file_label                  = dataset_label.lower().replace(' ','_')
# detailed_performance_report = report_path + file_label + '_detailed_performance_report.txt'

print(f"Year:                        {year}")
print(f"Clean File:                  {clean_file}")
# print(f"Performance Report:          {performance_report}")
# print(f"Detailed Performance Report: {detailed_performance_report}")

Year:                        2021
Clean File:                  data/brfss_2021_clean.parquet.gzip


In [3]:
# Read final cleaned dataset from parquet file
df = pd.read_parquet(clean_file, engine="fastparquet")

In [4]:
diabetes_labels = df.columns

In [5]:
df.shape

(247076, 37)

---

## 2. Prepare the dataset for analysis

- Split the dataset into features and labels.
- Split the dataset into training and testing sets.
- Scale the dataset

---

In [6]:
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

target = 'diabetes'
# Dictionary defining modification to be made to the base dataset
operation_dict = {  'target_column'     :  target,
                    'convert_to_binary' :  True,
                    'scaler'            : 'standard', # options: none, standard, minmax
                    'random_sample'     : 'undersample'      # options: none, undersample, oversample
                    }

# This insures that df if not modified during the call to modify_base_dataset()
df_modified = df.copy()

# Modify the base dataset
# data is returned where: X_train, X_test, y_train, y_test = data
data = mlanlys.modify_base_dataset(df_modified, operation_dict)

Base Dataset Modifications in Process
-------------------------------------
**Operation:target_column  diabetes
**Operation:convert_to_binary  True
  -- Converting dataset to binary (0,1) from (0,1,2)


****Cleaning Feature: diabetes
  Initial Unique features in [diabetes]:  [0. 1. 2.]
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: ********* NO Parameters were specified *********
  FINAL Unique features in [diabetes]:  [0. 1.]
**Operation:scaler  standard
  -- Performing train_test_split on dataframe with target:'diabetes'
     -- Run automatically before scalar or random_sample operations
  -- Performing StandardScaler on X_train: Updates X_train, y_test
**Operation:random_sample  undersample
  -- Performing RandomUnderSampler on X_train, y_train: Updates X_train, y_train

Dataframe, Train Test Summary
-----------------------------
Dataframe: (247076, 37)  Data:4, X_train:185307, y_train:185307, X_test:61769, y_test:61769
ValueCou

In [8]:
# Print some statistics about the original df and the modified dataframe
print(f"Original Dataframe")
print(f"------------------")
print(f"df.shape: {df.shape}")
print(f"df[{target}].value_counts:  {df[target].value_counts()}")

print(f"\nModified Dataframe")
print(f"------------------")
print(f"df_modified.shape: {df_modified.shape}")
print(f"df_modified[{target}].value_counts:  {df_modified[target].value_counts()}")

Original Dataframe
------------------
df.shape: (247076, 37)
df[diabetes].value_counts:  diabetes
0.0    208389
2.0     33033
1.0      5654
Name: count, dtype: int64

Modified Dataframe
------------------
df_modified.shape: (247076, 37)
df_modified[diabetes].value_counts:  diabetes
0.0    214043
1.0     33033
Name: count, dtype: int64


In [9]:
X_train, X_test, y_train, y_test = data
print(f"Dataframe: {df_modified.shape}  Data:{len(data)}, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")

Dataframe: (247076, 37)  Data:4, X_train:49462, y_train:49462, X_test:61769, y_test:61769


In [10]:
y_train.value_counts()

diabetes
0.0    24731
1.0    24731
Name: count, dtype: int64

In [11]:
y_test.value_counts()

diabetes
0.0    53467
1.0     8302
Name: count, dtype: int64

---

## 3. Optimization

---

#### 3.1 Pre-optimization metric results

- The summary report of the metrics for all pre-optimization runs is here:  [performance_report.txt](reports/performance_report.txt)
- The details of the runs are contained in these file:
    - [base_dataset_detailed_performance_report.txt](reports/base_dataset_detailed_performance_report.txt)
    - [binary_dataset_detailed_performance_report.txt](reports/binary_dataset_detailed_performance_report.txt)
    - [minmax_scaled_dataset_detailed_performance_report.txt](reports/minmax_scaled_dataset_detailed_performance_report.txt)
    - [randomoversample_dataset_detailed_performance_report.txt](reports/randomoversample_dataset_detailed_performance.txt)
    - [cluster_dataset_detailed_performance_report.txt](reports/cluster_dataset_detailed_performance_report.txt)
    - [randomundersampled_dataset_detailed_performance_report.txt](reports/randomundersampled_dataset_detailed_performance_report.txt)
    - [minmax_scaled_dataset_detailed_performance_report.txt](reports/minmax_scaled_dataset_detailed_performance_report.txt)
    - [smoteen_dataset_detailed_performance_report.txt](reports/smoteen_dataset_detailed_performance_report.txt)
    - [standard_scaled_dataset_detailed_performance_report.txt](reports/standard_scaled_dataset_detailed_performance_report.txt)
    - [smote_dataset_detailed_performance_report.txt](reports/smote_dataset_detailed_performance_report.txt)

#### 3.2 Optimization Dataset used

**Note:**  Modify the dataset as desired in Section 2.
<br><br>
Currently the dataset uses the Base dataset for 2015 with the following modifications:
- **Target converted to Binary**:  (0,1) from (0,1,2).
    - Base:  0: No diabetes, 1: Pre-diabetes, 2: have diabetes
    - binary: 0: No diabetes/pre-diabetes, 1: have diabetes
- Scaled the data with **StandardScaler**
- Resampled the data with **RandomUnderSampler**


#### 3.2 Optimization Analysis

In [12]:
# Note:  Modify the dataset as desired in Section 2.
# Currently the dataset uses the Base dataset for 2015 with the following modifications

In [13]:
# Dataset summary:
X_train, X_test, y_train, y_test = data
print(f"Dataset Lens, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")
print(f"y_train value_counts: {y_train.value_counts()}")
print(f"y_test value_counts: {y_test.value_counts()}")


Dataset Lens, X_train:49462, y_train:49462, X_test:61769, y_test:61769
y_train value_counts: diabetes
0.0    24731
1.0    24731
Name: count, dtype: int64
y_test value_counts: diabetes
0.0    53467
1.0     8302
Name: count, dtype: int64


##### 3.2.1 Perform Optimization:

- Datamodel:  2015 Diabetes Data Set
- Optimization approaches: DecisionTreeClassifier RandomizedSearchCV


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

import numpy as np

# Define the parameter grid
param_distributions = {
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)],  # Depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider for the best split
}

# parameters = {'criterion':['gini','entropy'],
#             'max_depth':poisson(mu=2,loc=2),
#             'min_samples_split':uniform(),
#             'max_leaf_nodes':poisson(mu=4,loc=3)}

# Initialize the Decision Tree Classifier
dtc = DecisionTreeClassifier()

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    dtc,
    param_distributions,
    cv=5,        # 5-fold cross-validation
    n_iter=100,  # Number of parameter settings that are sampled
    verbose=3,   # Verbosity mode
    n_jobs=-1,   # Use all available cores
    random_state=42  # Seed for reproducibility
)

# Fit the model
random_search.fit(X_train, y_train)

# Best model
best_dtc = random_search.best_estimator_

# Make predictions on the test set
y_pred_dtc = best_dtc.predict(X_test)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[CV 1/5] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=10;, score=0.661 total time=   0.1s
[CV 4/5] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=10;, score=0.680 total time=   0.2s
[CV 2/5] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=10;, score=0.670 total time=   0.2s
[CV 3/5] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=10;, score=0.671 total time=   0.2s
[CV 2/5] END max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5;, score=0.725 total time=   0.1s
[CV 1/5] END max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=2;, score=0.672 total time=   0.2s
[CV 5/5] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=10;, score=0.668 total time=   0.2s[CV 1/5] END max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5;, score=0.718 total time=   0.1s

[CV 4/5] END max_depth=10, max_features=log

In [24]:
# Summarize the Performance of the Best Classifier
print(f"---------------------------------------------")
print(f"Best Parameters for Decision Tree Classifier: {random_search.best_params_}")
print(f"---------------------------------------------\n\n")

best_data = X_test, y_test, y_pred_dtc
mlanlys.model_performance_metrics(best_dtc, best_data, "Best Decision Tree Classifier")

# Evaluate the classifier
mse_dtc = mean_squared_error(y_test, y_pred_dtc)

print(f"Mean Squared Error of Decision Tree Classifier: {mse_dtc}")

---------------------------------------------
Best Parameters for Decision Tree Classifier: {'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 10}
---------------------------------------------


------------------------------------------------------------------------
---------- Best Decision Tree Classifiering Data Performance
------------------------------------------------------------------------
Confusion Matrix
[[35531 17936]
 [ 1777  6525]]

-----------------------
Best Decision Tree Classifier score: 0.6808593307322444
Balanced Accuracy Score: 0.7252479681392824
ROC AUC Score: 0.7900964322506635

-----------------------
Classification Report
              precision    recall  f1-score   support

         0.0       0.95      0.66      0.78     53467
         1.0       0.27      0.79      0.40      8302

    accuracy                           0.68     61769
   macro avg       0.61      0.73      0.59     61769
weighted avg       0.86      0.68     

In [23]:
# Model without optimization
X_train, X_test, y_train, y_test = data

model = DecisionTreeClassifier()
# Train the model
model = model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

test_data = [ X_test, y_test, y_test_pred]

mlanlys.model_performance_metrics(model, test_data, "Original Decision Tree Classifier" )

# Evaluate the classifier
mse_orig = mean_squared_error(y_test, y_test_pred)

print(f"Mean Squared Error of Decision Tree Classifier: {mse_orig}")

------------------------------------------------------------------------
---------- Original Decision Tree Classifiering Data Performance
------------------------------------------------------------------------
Confusion Matrix
[[35566 17901]
 [ 2848  5454]]

-----------------------
Original Decision Tree Classifier score: 0.6640871634638734
Balanced Accuracy Score: 0.6610727433209354
ROC AUC Score: 0.6610727433209355

-----------------------
Classification Report
              precision    recall  f1-score   support

         0.0       0.93      0.67      0.77     53467
         1.0       0.23      0.66      0.34      8302

    accuracy                           0.66     61769
   macro avg       0.58      0.66      0.56     61769
weighted avg       0.83      0.66      0.72     61769

Mean Squared Error of Decision Tree Classifier: 0.3359128365361265


---

## 4. Conclusions

- The confusion matrix shows more TP/TN values and Fewer FP/FN values 
- Classification accuracy increased from .66 to .68
- Balanced accuracy increased from .66 to .72
- ROC AUC Score increased from .66 to .78

**Summary:  Decision Tree Classifiering Performance Improved**

---